In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import networkx as nx
from IPython.display import display, clear_output
from scipy import signal
from scipy import fftpack
from numpy.fft import fft, fftfreq
import os
import pandas as pd

# import custom module
import SIRS

In [2]:
# needed only if you made some changes at SIRS.py and want to import the updated version of the module
from importlib import reload
reload(SIRS)

<module 'SIRS' from '/home/clara/OneDrive/Università/Terzo Semestre/Life Data Epidemiology/LDE_project/LifeDataEpidemiology/SIRS.py'>

# STARTING TO EXPLORE MOBILITY: changing probability of travel p_mob

We fixed the epidemic parameters in a "critical" point at which most of the times the epidemics does not survive and dies out after some iterations. The topology of the two networks is also kept fixed. We now explore the role of the mobility parameter p_mob.

In [13]:
#mobility and topology parameters
N = 1000
I_sf_init = 5
I_er_init = 0
mean_degree = 4
eps = 0.1 

Basic Reproductive Ratio (taking into account topology): 0.4166666666666667
β (taking into account topology): 0.0625


In [22]:
beta_list = [0.3]
gamma_list = [0.016, 0.02]
mu = 0.15

p_mob_list = np.linspace(0.03,0.2,20)

#print(gamma_list)
#print(beta_list)
#print(p_mob_list)

[0.016, 0.02]
[0.3]
[0.03       0.03894737 0.04789474 0.05684211 0.06578947 0.07473684
 0.08368421 0.09263158 0.10157895 0.11052632 0.11947368 0.12842105
 0.13736842 0.14631579 0.15526316 0.16421053 0.17315789 0.18210526
 0.19105263 0.2       ]


In [ ]:
for beta in beta_list:
    
    
    for gamma in gamma_list:
    
    
        beta = beta/mean_degree
        infection_params = dict(beta=beta, mu=mu, gamma=gamma)
        
        #R_0 = beta/mu
    
        #print("Basic Reproductive Ratio (taking into account topology):", R_0)
        #print("\u03B2 (taking into account topology):", beta)
        
        
        UP_directory_name = str(beta)+"_"+str(mu)+"_"+str(gamma) 
        os.system("mkdir "+UP_directory_name)

        for p_mob in p_mob_list:

            
            
            #print("P_mob:", p_mob)

            
            #saving stuff for future analysis
            directory_name = UP_directory_name+"/pmob_"+"{:3f}".format(p_mob)
            os.system("mkdir "+directory_name)

            #saving epidemics, topology and mobility parameters

            params=dict(N=N, I_sf_init=I_sf_init, I_er_nit=I_er_init, p_mob=p_mob, mean_degree=mean_degree,
                        eps=eps, **infection_params)

            parameters = pd.DataFrame(params, index=[0])
            save=directory_name+"/parameters.csv"
            parameters.to_csv(save)



            #STARTING SIMULATION
            n_runs = 30
            n_iter = 1000


            S_tot_er=0*(n_iter+1)
            I_tot_er=0*(n_iter+1)
            R_tot_er=0*(n_iter+1)
            S_tot_sf=0*(n_iter+1)
            I_tot_sf=0*(n_iter+1)
            R_tot_sf=0*(n_iter+1)


            for run in range(n_runs):  


                # prepare systems
                state_sf_init, state_er_init, variables_net_sf, variables_net_er = SIRS.prepare_two_sys(N, I_sf_init, I_er_init, 
                                                                                              p_mob, mean_degree)

                #saving initial states
                save = directory_name + "/state_sf_init_"+str(run)+".txt"
                np.savetxt(save, state_sf_init)

                save = directory_name + "/state_er_init_"+str(run)+".txt"
                np.savetxt(save, state_sf_init)

                #saving initial variables: we just save the IDs of the travelling nodes and the initial network 
                #(as adjacency matrix)
                save = directory_name + "/A_sf_"+str(run)+".txt"
                np.savetxt(save, variables_net_sf["A_sf"])
                save = directory_name + "/A_er_"+str(run)+".txt"
                np.savetxt(save, variables_net_er["A_er"])


                save = directory_name + "/travellers_sf_"+str(run)+".txt"
                np.savetxt(save, variables_net_sf["travellers_sf"])
                save = directory_name + "/travellers_er_"+str(run)+".txt"
                np.savetxt(save, variables_net_er["travellers_er"])


                #print("Iteration:", run)

                state_sf = np.copy(state_sf_init)
                state_er = np.copy(state_er_init)

                S_sf = [N - I_sf_init]
                I_sf = [I_sf_init]
                R_sf = [0]
                S_er = [N - I_er_init]
                I_er = [I_er_init]
                R_er = [0]
                t_vec = []
                for i in range(n_iter):
                    state_sf, state_er = SIRS.two_sys_full_SIRS_step(state_sf, state_er, **variables_net_sf, 
                                                            **variables_net_er, **infection_params)
                    S_sf.append(state_sf[:,0].sum())
                    I_sf.append(state_sf[:,1].sum())
                    R_sf.append(state_sf[:,2].sum())
                    S_er.append(state_er[:,0].sum())
                    I_er.append(state_er[:,1].sum())
                    R_er.append(state_er[:,2].sum())
                    #t_vec.append(time.time()-start)

                #tot_time = time.time()-start
                #print("Total time elapsed: %.2f s"%tot_time)
                #print("Time per iteration: %.4f s"%(tot_time/n_iter))

                S_sf = np.array(S_sf)
                I_sf = np.array(I_sf)
                R_sf = np.array(R_sf)
                S_er = np.array(S_er)
                I_er = np.array(I_er)
                R_er = np.array(R_er)
                t_vec = np.array(t_vec)

                #save timeseries of SIR
                SIR_sf = np.vstack((S_sf, I_sf, R_sf)).T
                save = directory_name+"/SIR_sf_"+str(run)+".txt"
                np.savetxt(save, SIR_sf)
                SIR_er = np.vstack((S_er, I_er, R_er)).T
                save = directory_name+"/SIR_er_"+str(run)+".txt"
                np.savetxt(save, SIR_er)

                S_tot_sf+=S_sf
                I_tot_sf+=I_sf
                R_tot_sf+=R_sf
                S_tot_er+=S_er
                I_tot_er+=I_er
                R_tot_er+=R_er

                '''indexes = np.arange(n_iter+1)
                fig, ax = plt.subplots(1,2, figsize=(15,8))
                ax[0].plot(indexes, S_sf, label = r'$S_{sf}$')
                ax[0].plot(indexes, I_sf, label = r'$I_{sf}$')
                ax[0].plot(indexes, R_sf, label = r'$R_{sf}$')
                ax[0].plot(indexes, S_sf+I_sf+R_sf, label = 'tot')
                ax[0].set_xlabel('Number of iterations', fontsize = 16)
                ax[0].set_ylabel('Number of individuals', fontsize = 16)
                ax[0].set_title("Scale-free network", fontsize = 16)
                ax[0].legend(fontsize=13, loc='upper right')

                ax[1].plot(indexes, S_er, label = r'$S_{er}$')
                ax[1].plot(indexes, I_er, label = r'$I_{er}$')
                ax[1].plot(indexes, R_er, label = r'$R_{er}$')
                ax[1].plot(indexes, S_er+I_er+R_er, label = 'tot')
                ax[1].set_title("Erdosh-Renyi network", fontsize = 16)
                ax[1].set_xlabel('Number of iterations', fontsize = 16)
                ax[1].set_ylabel('Number of individuals', fontsize = 16)
                ax[1].legend(fontsize=13, loc='upper right')

                plt.suptitle(r"p_mob = {:.3f}; $\beta= {:.3f}; \mu= {:.3f}; \gamma= {:.3f}$".format(p_mob, mu, beta, gamma), fontsize=20)

                save=directory_name+"/run_"+str(run)+".png"
                fig.savefig(save)
                plt.close()'''

            indexes = np.arange(n_iter+1)
            fig, ax = plt.subplots(1,2, figsize=(15,8))
            ax[0].plot(indexes, S_tot_sf/n_runs, label = r'$S_{sf}$')
            ax[0].plot(indexes, I_tot_sf/n_runs, label = r'$I_{sf}$')
            ax[0].plot(indexes, R_tot_sf/n_runs, label = r'$R_{sf}$')
            ax[0].plot(indexes, (S_tot_sf+I_tot_sf+R_tot_sf)/n_runs, label = 'tot')
            ax[0].set_xlabel('Number of iterations', fontsize = 16)
            ax[0].set_ylabel('Number of individuals', fontsize = 16)
            ax[0].set_title("Scale-free network", fontsize = 16)
            ax[0].legend(fontsize=13, loc='upper right')

            ax[1].plot(indexes, S_tot_er/n_runs, label = r'$S_{er}$')
            ax[1].plot(indexes, I_tot_er/n_runs, label = r'$I_{er}$')
            ax[1].plot(indexes, R_tot_er/n_runs, label = r'$R_{er}$')
            ax[1].plot(indexes, (S_tot_er+I_tot_er+R_tot_er)/n_runs, label = 'tot')
            ax[1].set_title("Erdosh-Renyi network", fontsize = 16)
            ax[1].set_xlabel('Number of iterations', fontsize = 16)
            ax[1].set_ylabel('Number of individuals', fontsize = 16)
            ax[1].legend(fontsize=13, loc='upper right')
            
            fig.tight_layout()

            plt.suptitle(r"Mean on {} runs, p_mob = {:.3f}; $\beta= {:.3f}; \mu= {:.3f}; \gamma= {:.3f}$".format(n_runs, p_mob, mu, beta, gamma), fontsize=20)


            save = directory_name+"/total_"+str(p_mob)+".png"
            fig.savefig(save)
            plt.close()